<a href="https://colab.research.google.com/github/adimis-ai/Query-Classifier-Extractive-Q-A/blob/main/Query_Classifier_Extractive_Q%26A_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest main of Haystack (Colab)
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

# Install these to allow pipeline visualization
!apt install libgraphviz-dev
!pip install pygraphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-nllplzek/farm-haystack_f4a10d8cf496428bbc084cdb02de1311
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-nllplzek/farm-haystack_f4a10d8cf496428bbc084cdb02de1311
  Resolved https://github.com/deepset-ai/haystack.git to commit db6e5754cd71dd822f466bebe3882dc61de00c44
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.4/381.4 kB 9.

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.utils import (
    print_answers,
    print_documents,
    fetch_archive_from_http,
    convert_files_to_docs,
    clean_wiki_text,
    launch_es,
)
from haystack.pipelines import Pipeline
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader, TransformersQueryClassifier

# DATA - 517 Wikipedia articles for Game of Thrones
doc_dir = "gdrive/MyDrive/wiki_gameofthrones_txt1/"

# convert files to dicts containing documents that can be indexed to our datastore
got_docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Initialize DocumentStore and index documents
# launch_es() # Uncomment this line for local Elasticsearch
document_store = ElasticsearchDocumentStore()
document_store.delete_documents()
document_store.write_documents(got_docs)


INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/130_Game_of_Thrones_title_sequence.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/102_The_Princess_and_the_Queen.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/347_Game_of_Thrones__season_2_.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/346_Ygritte.txt
INFO:haystack.utils.preprocessing:Converting gdrive/MyDrive/wiki_gameofthrones_txt1/37_Joffrey_Baratheon

In [ ]:
# Initialize sparse retriever for keyword queries
bm25_retriever = BM25Retriever(document_store=document_store)

# Initialize dense retriever for question/statement queries
embedding_retriever = EmbeddingRetriever(
    document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1"
)
document_store.update_embeddings(embedding_retriever, update_existing_embeddings=False)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/multi-qa-mpnet-base-dot-v1


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/74 [00:00<?, ?it/s]

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
# Here we build the pipeline
transformer_keyword_classifier = Pipeline()

transformer_keyword_classifier.add_node(
    component=TransformersQueryClassifier(), name="QueryClassifier", inputs=["Query"]
)

transformer_keyword_classifier.add_node(
    component=embedding_retriever, name="EmbeddingRetriever", inputs=["QueryClassifier.output_1"]
)

transformer_keyword_classifier.add_node(
    component=bm25_retriever, name="BM25Retriever", inputs=["QueryClassifier.output_2"]
)

transformer_keyword_classifier.add_node(
    component=reader, name="QAReader", inputs=["BM25Retriever", "EmbeddingRetriever"]
)


# Useful for framing headers
equal_line = "=" * 30

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
QUERY = input("Enter your query:  ")

Enter your query:  why is arya stark such an unsual character


In [ ]:
# Run only the dense retriever on the full sentence query
res_1 = transformer_keyword_classifier.run(query=QUERY)
print(f"\n\n{equal_line}\nQUESTION QUERY RESULTS\n{equal_line}")
print_answers(res_1, details="minimum")
print("\n\n")

# Run only the sparse retriever on a keyword based query
res_2 = transformer_keyword_classifier.run(query=QUERY)
print(f"\n\n{equal_line}\nKEYWORD QUERY RESULTS\n{equal_line}")
print_answers(res_2, details="minimum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:28<00:00, 28.87s/ Batches]



QUESTION QUERY RESULTS

Query: why is arya stark such an unsual character
Answers:
[   {   'answer': 'she has grown feral',
        'context': 'd by her long-lost direwolf Nymeria. Nymeria recognizes '
                   'Arya, but she has grown feral and ignores Arya when she '
                   'asks her to come North with her.\n'
                   'Arriv'},
    {   'answer': 'she does not want the life of a lady',
        'context': 'egitimised as a Baratheon by Daenerys. Arya declines, as '
                   'she does not want the life of a lady. Sansa and Arya tell '
                   "Jon they don't trust Daenerys, but "},
    {   'answer': 'Arya ran off with her wolf',
        'context': ' in order to protect both Joffrey and her sister Arya.  '
                   "Since Arya ran off with her wolf to save it, Sansa's wolf "
                   'is killed instead, estranging the St'},
    {   'answer': "she did so to try and save Ned's life",
        'context': 'nfronts Sans

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:28<00:00, 28.05s/ Batches]



KEYWORD QUERY RESULTS

Query: why is arya stark such an unsual character
Answers:
[   {   'answer': 'she has grown feral',
        'context': 'd by her long-lost direwolf Nymeria. Nymeria recognizes '
                   'Arya, but she has grown feral and ignores Arya when she '
                   'asks her to come North with her.\n'
                   'Arriv'},
    {   'answer': 'she does not want the life of a lady',
        'context': 'egitimised as a Baratheon by Daenerys. Arya declines, as '
                   'she does not want the life of a lady. Sansa and Arya tell '
                   "Jon they don't trust Daenerys, but "},
    {   'answer': 'Arya ran off with her wolf',
        'context': ' in order to protect both Joffrey and her sister Arya.  '
                   "Since Arya ran off with her wolf to save it, Sansa's wolf "
                   'is killed instead, estranging the St'},
    {   'answer': "she did so to try and save Ned's life",
        'context': 'nfronts Sansa